<a href="https://colab.research.google.com/github/ncn-foreigners/job-ads-classifier/blob/main/job_classification_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Clone repository

In [1]:
!git clone https://github.com/ncn-foreigners/job-ads-classifier.git

Cloning into 'job-ads-classifier'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 77 (delta 16), reused 39 (delta 2), pack-reused 0
Unpacking objects: 100% (77/77), done.


## Colab settings


In [2]:
from google.colab.data_table import DataTable
DataTable.max_columns = 60

## Set up environment

Install required modules

In [3]:
!pip install virtualenv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.8 MB 9.3 MB/s 
     |████████████████████████████████| 468 kB 52.1 MB/s 


In [4]:
!virtualenv classifier

created virtual environment CPython3.7.15.final.0-64 in 1152ms
  creator CPython3Posix(dest=/content/classifier, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==22.2.2, setuptools==65.3.0, wheel==0.37.1
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [6]:
!source classifier/bin/activate

Upgrade pip

In [7]:
!pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Install torch for this machine

In [8]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Install required packages

In [9]:
!pip install -r ./job-ads-classifier/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.0/73.0 kB 9.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.5/541.5 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 52.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.7/361.7 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.4/660.4 kB 51.3 MB/s eta 0:00:00
    

## Working with trained models

Load required modules

In [2]:
import sys
sys.path.insert(1, '/content/job-ads-classifier')

In [3]:
# Numpy, pandas i matplotlib jako pomocnicze biblioteki
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from job_offers_classifier.load_save import load_to_df, save_obj, load_obj
from job_offers_classifier.job_offers_utils import create_hierarchy, fix_class_str, remove_classes, filter_classes, remap_classes
from job_offers_classifier.job_offers_classfier import *
from sklearn.model_selection import train_test_split
import torch
from torchmetrics import MetricCollection, Recall, Accuracy, ConfusionMatrix

OSError: ignored

Load functions and hierarchy

Example data -- job offers from Public Sercice

In [47]:
kprm_df=pd.read_xml("https://nabory.kprm.gov.pl/pls/serwis/app.xml", xpath = "./new/oferta")
kprm_df = kprm_df.replace("\\<\\!\\[CDATA\\[ ", "", regex=True)
kprm_df = kprm_df.replace(" \\]\\]\\>", "", regex=True)
kprm_df = kprm_df.replace(r'<[^<>]*>', '', regex=True)
kprm_df = kprm_df.replace('&nbsp;', ' ', regex=True)
kprm_df = kprm_df.replace('&dash;', ' ', regex=True)
kprm_df = kprm_df.replace('&oacute;', ' ', regex=True)
kprm_df = kprm_df.replace('\r', ' ', regex=True)
kprm_df[["unid", "nazwa_firmy", "stanowisko", "do_spraw", "poledodatkowelista1", "poledodatkowelista3"]].head(n=2)

,unid,nazwa_firmy,stanowisko,do_spraw,poledodatkowelista1,poledodatkowelista3
0,108581,Ministerstwo Spraw Zagranicznych w Warszawie,inspektor,ekspedycji informacji niejawnych,Prowadzi dzienniki ewidencji materiałów niejaw...,Warunki dotyczące charakteru pracy na stanowis...
1,108580,Powiatowy Inspektorat Nadzoru Budowlanego w Op...,inspektor nadzoru budowlanego,kontroli,Przygotowywanie i opracowywanie projektów decy...,Praca w siedzibie Inspektoratu przy wykorzysta...


In [48]:
kprm_df.poledodatkowelista1[1]

'Przygotowywanie i opracowywanie projektów decyzji, postanowień i innych pism z zakresu właściwości Powiatowego Inspektora Nadzoru Budowlanego.Prowadzenie postępowań administracyjnych.Prowadzenie kontroli budów, robót budowlanych i utrzymania obiektów budowlanych.Przyjmowanie zawiadomień o zamierzonym terminie rozpoczęcia robót budowlanych, o zakończeniu budowy oraz wniosków o udzielenie pozwolenia na użytkowanie.Prowadzenie ewidencji i przygotowywanie wymaganych sprawozdań.'

### Logistic regression model using tf-df matrix

### Transformer model